In [5]:
import pandas as pd 
from mylibrary.connections import cursor, conn, engine, Automapped_Base, session

sql = """
select * from tt_h.stations_journeytime_cw where tfl_message = 'ok' limit 3
"""
df = pd.read_sql(sql, conn)

In [83]:
import json 

for r in df.iterrows():
    index = r[0]
    row = r[1]

    j1 = row["tfl_response"]
    j2 = json.loads(j1)
    
    journeys = j2["journeys"]
    journey = sorted(journeys, key = lambda x:  x["duration"])[0]
    
    legs = journey["legs"]
    
    legs = remove_non_national_rail(legs)
    
    summary = summarise_legs(legs)
    travel_time_minutes = get_total_travel_time(legs) 
    
    lat_lng_from_legs(legs):
    
    
    df.loc[index, "hello"] 




In [47]:
# Remove any journeys at the end which are not national rail
# Initial assumption will be that all national rail legs are done
def remove_non_national_rail(legs):
    # Pop one at a time 
    while True:
        leg = legs.pop()
        if leg['mode']['id'] == "national-rail":
            break
    legs.append(leg)
    return legs


In [50]:
def summarise_legs(legs):
    final_string_elems = []
    for leg in legs:
        final_string_elems.append(leg["departurePoint"]["commonName"])
    final_string_elems.append(leg["arrivalPoint"]["commonName"])
    return " -> ".join(final_string_elems)

In [51]:
summarise_legs(legs)

'White Notley Rail Station -> Stratford (London) Rail Station'

In [65]:
from dateutil.parser import parse
def get_total_travel_time(legs):
    dt = parse(legs[0]["departureTime"])
    at = parse(legs[-1]["arrivalTime"])
    td = at-dt
    return td.total_seconds()/60
    
get_total_travel_time(legs)  


44.0

In [77]:
#need lat lng of final location
#need lat lng of initial location 
def lat_lng_from_legs(legs):
    d_lat = legs[0]["departurePoint"]["lat"]
    d_lon = legs[0]["departurePoint"]["lon"]
    a_lat = legs[-1]["arrivalPoint"]["lat"]
    a_lon = legs[-1]["arrivalPoint"]["lon"]
    
    return {"depart":{"lat": d_lat, "lng":d_lon}, "arrive":{"lat":a_lat, "lng":a_lon}}
